<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Віконні функції в SQL

**Віконні функції (window functions)** — це функції, які працюють по групі рядків (вікну), але при цьому **не згортають результат до одного рядка**, на відміну від `GROUP BY`.  
Вони дозволяють виконувати обчислення "поверх" даних, залишаючи повний набір рядків.

---

## Основні віконні функції

| Функція        | Призначення                                    |
| -------------- | ---------------------------------------------- |
| `ROW_NUMBER()` | Порядковий номер рядка в групі                 |
| `RANK()`       | Ранг із пропусками у випадку однакових значень |
| `DENSE_RANK()` | Ранг без пропусків                             |
| `LAG()`        | Попереднє значення                             |
| `LEAD()`       | Наступне значення                              |
| `SUM()`        | Накопичувальний підсумок                       |
| `AVG()`        | Середнє по вікну                               |

## ВІКОННА ФУНКЦіЯ SUM(...) OVER vs. GROUP BY

Уявімо таблицю `orders_log`:

| user\_uuid | order\_time | total\_uah |
| ---------- | ----------- | ---------- |
| u1         | 2024-01-10  | 300        |
| u1         | 2024-01-15  | 400        |
| u2         | 2024-01-12  | 100        |
| u2         | 2024-01-22  | 500        |


Результат `SUM(...) OVER`  

| user\_uuid | order\_time | total\_uah | cumulative\_total |
| ---------- | ----------- | ---------- | ----------------- |
| u1         | 2024-01-10  | 300        | 300               |
| u1         | 2024-01-15  | 400        | 700               |
| u2         | 2024-01-12  | 100        | 100               |
| u2         | 2024-01-22  | 500        | 600               |

Результат `GROUP BY`

| user\_uuid | total\_by\_user |
| ---------- | --------------- |
| u1         | 700             |
| u2         | 600             |

## Принципи написання віконних функцій

### Синтаксис

```sql
SELECT column,
       функція(...) OVER (
        ...
       )
FROM таблиця;
```

```sql
ФУНКЦИЯ(...) OVER (
    [PARTITION BY розбиття_на_підгрупи]
    [ORDER BY порядок_рядків_у_вікні]
    [ROWS BETWEEN ... AND ...]
)
```

| Елемент            | Обов’язково? | Призначення                                                           |
| ------------------ | ------------ | --------------------------------------------------------------------- |
| `ФУНКЦІЯ(...)`     | ✅ Так        | Віконна функція (`SUM`, `ROW_NUMBER`, `LAG` тощо)                     |
| `OVER (...)`       | ✅ Так        | Обов’язковий блок, який робить функцію віконною                       |
| `PARTITION BY ...` | ❌ Ні         | Розділення на підгрупи (наприклад, за `user_id`)                      |
| `ORDER BY ...`     | ❌ Ні         | Визначає порядок рядків у межах кожної групи                          |
| `ROWS BETWEEN ...` | ❌ Ні         | Уточнює вікно: які рядки з відсортованої групи беруться до розрахунку |


### Компоненти віконної функції

#### `ФУНКЦІЯ(...)`

Це агрегатна або віконна функція, що застосовується до вікна рядків (не до всієї таблиці). Наприклад:  


* `SUM()`, `AVG()`, `COUNT()`
* `ROW_NUMBER()`, `RANK()`, `LAG()`, `LEAD()`

#### `OVER (...)` — "вікно", за яким працюватиме функція

Ключова відмінність від звичайної агрегації: **результат залишається у кожному рядку**, а не згортається.

##### `PARTITION BY`

* Ділить дані на **групи** (розбиття).
* Кожна група обробляється окремо.

📌 Аналог `GROUP BY`, але результат **не згортається**, а залишається построчно.

**Приклад:**

```sql
SUM(total_uah) OVER (PARTITION BY user_uuid)
```

Рахує суму для кожного користувача по всіх його рядках.

---

##### `ORDER BY`

* Сортує рядки **в межах кожної групи (partition)**.
* Обов’язково для функцій, яким важливий **порядок**: `ROW_NUMBER()`, `RANK()`, `LAG()`, `LEAD()`, `CUMULATIVE SUM`.

📌 Якщо **не вказати `ORDER BY`**, то:

* Для `LAG`, `LEAD`, `ROW_NUMBER` та інших **порядок буде невизначений** — результат може бути некоректним.
* Для `SUM(...) OVER (PARTITION BY ...)` без `ORDER BY` — буде сума **по всій групі**, але без накопичення.

---

##### `ROWS BETWEEN ... AND ...` — діапазон рядків

Визначає, **який діапазон рядків** всередині "вікна" включити до розрахунку.

**Загальна форма:**

```sql
ROWS BETWEEN <початок_вікна> AND <кінець_вікна>
```

---

**Можливі значення**

| Значення              | Що означає                         |
| --------------------- | ---------------------------------- |
| `UNBOUNDED PRECEDING` | Початок вікна (перший рядок групи) |
| `n PRECEDING`         | `n` рядків перед поточним          |
| `CURRENT ROW`         | Поточний рядок                     |
| `n FOLLOWING`         | `n` рядків після поточного         |
| `UNBOUNDED FOLLOWING` | До кінця вікна                     |

---

**Приклади комбінацій**

| Синтаксис                                                  | Що робить                                                      |
| ---------------------------------------------------------- | -------------------------------------------------------------- |
| `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`         | Накопичувальна сума від початку до поточного рядка             |
| `ROWS BETWEEN 3 PRECEDING AND CURRENT ROW`                 | Скользяче вікно з 4 рядків: поточний та 3 попередніх           |
| `ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING`                 | Сусіди навколо рядка (3 рядки: до, поточний, після)            |
| `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`         | Від поточного рядка до кінця групи                             |
| `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` | Вся група цілком (еквівалентно `PARTITION BY`, але з ORDER BY) |
| `ROWS BETWEEN CURRENT ROW AND CURRENT ROW`                 | Лише поточний рядок (за замовчуванням, якщо нічого не вказано) |

📌 Якщо вказано `ORDER BY`, але не вказано `ROWS`, за замовчуванням:

```sql
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
```

---

##### Приклад

```sql
SELECT user_uuid,
       order_time,
       total_uah,
       SUM(total_uah) OVER (
           PARTITION BY user_uuid
           ORDER BY order_time
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS cumulative_total
FROM orders_log;
```

| Елемент                  | Пояснення                                        |
| ------------------------ | ------------------------------------------------ |
| `SUM(total_uah)`         | Функція: обчислюємо накопичувальну суму          |
| `PARTITION BY user_uuid` | Для кожного користувача окремо                   |
| `ORDER BY order_time`    | Рахуємо **у хронологічному порядку** замовлень   |
| `ROWS BETWEEN ...`       | Накопичення: від першого до поточного замовлення |
| `AS cumulative_total`    | Назва нового стовпця з результатом               |

#### Типові помилки

| Помилка                                                        | Що піде не так                                                                 |
| -------------------------------------------------------------- | ------------------------------------------------------------------------------ |
| **Не вказали `ORDER BY` для `ROW_NUMBER()` або `LAG()`**       | Порядок буде випадковим                                                        |
| **Не вказали `PARTITION BY`, а він був потрібен**              | Всі користувачі будуть в одній групі                                           |
| **Не вказали `ROWS` в агрегатних функціях (`SUM()`, `AVG()`)** | У PostgreSQL рахуватиме від початку до поточного рядка, але краще вказати явно |

---

#### Як правильно писати віконні функції

1. **Визнач групи**, за якими працює логіка (`PARTITION BY`)
2. **Визнач порядок** всередині групи (`ORDER BY`)
3. Для агрегатів — **уточни вікно рядків** (`ROWS BETWEEN`)
4. Переконайся, що **функція підходить для поточного завдання**


## Віконні функції

### Тестова таблиця `orders_log`

| user\_id | order\_time | total_uah |
| -------- | ----------- | ------ |
| u1       | 2024-01-01  | 100    |
| u1       | 2024-01-10  | 150    |
| u1       | 2024-01-20  | 200    |
| u2       | 2024-01-05  | 300    |
| u2       | 2024-02-01  | 100    |

### 1. `SUM(...) OVER (PARTITION BY ...)` — сума по користувачу

```sql
SELECT user_uuid, order_time, total_uah,
      SUM(total_uah) OVER (
        PARTITION BY user_uuid
      )
FROM orders_log;
```

**Що відбувається:**

* Кожному рядку присвоюється сума всіх `total_uah` для цього `user_uuid`.
* Порядок не має значення — це просто агрегат без сортування.

**Результат:**

| user\_uuid | order\_time | total\_uah | sum |
| ---------- | ----------- | ---------- | --- |
| u1         | 2024-01-01  | 100        | 450 |
| u1         | 2024-01-10  | 150        | 450 |
| u1         | 2024-01-20  | 200        | 450 |
| u2         | 2024-01-05  | 300        | 400 |
| u2         | 2024-02-01  | 100        | 400 |


### 2. `SUM(...) OVER (PARTITION BY ... ORDER BY ...)` — кумулятивна сума

```sql
SELECT user_uuid, order_time, total_uah,
      SUM(total_uah) OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      )
FROM orders_log;
```

**Що відбувається:**

* Спочатку дані розбиваються по `user_uuid`.
* Потім у кожній групі сортуються за `order_time`.
* Обчислюється наростаюча сума.

**Результат:**

| user\_uuid | order\_time | total\_uah | sum |
| ---------- | ----------- | ---------- | --- |
| u1         | 2024-01-01  | 100        | 100 |
| u1         | 2024-01-10  | 150        | 250 |
| u1         | 2024-01-20  | 200        | 450 |
| u2         | 2024-01-05  | 300        | 300 |
| u2         | 2024-02-01  | 100        | 400 |


### 3. `LAG()` та `LEAD()` — попереднє та наступне значення

```sql
SELECT user_uuid, order_time, total_uah,
      LAG(total_uah) OVER (
          PARTITION BY user_uuid
          ORDER BY order_time
      ),
      LEAD(total_uah) OVER (
          PARTITION BY user_uuid
          ORDER BY order_time
      )
FROM orders_log;
```

**Пояснення:**

* `LAG(...)` — повертає значення попереднього рядка всередині групи.
* `LEAD(...)` — повертає значення наступного рядка всередині групи.

**Результат:**

| user\_uuid | order\_time | total\_uah | lag  | lead |
| ---------- | ----------- | ---------- | ---- | ---- |
| u1         | 2024-01-01  | 100        | null | 150  |
| u1         | 2024-01-10  | 150        | 100  | 200  |
| u1         | 2024-01-20  | 200        | 150  | null |
| u2         | 2024-01-05  | 300        | null | 100  |
| u2         | 2024-02-01  | 100        | 300  | null |


### 4. `ROW_NUMBER()` — порядковий номер у групі

```sql
SELECT user_uuid, order_time, total_uah,
      ROW_NUMBER() OVER (
        PARTITION BY user_uuid ORDER BY order_time
      )
FROM orders_log;
```

**Що робить:**

* Пронумеровує рядки **всередині кожної групи по `user_uuid`**.
* Порядок задається за датою.

**Результат:**

| user\_uuid | order\_time | total\_uah | row\_number |
| ---------- | ----------- | ---------- | ----------- |
| u1         | 2024-01-01  | 100        | 1           |
| u1         | 2024-01-10  | 150        | 2           |
| u1         | 2024-01-20  | 200        | 3           |
| u2         | 2024-01-05  | 300        | 1           |
| u2         | 2024-02-01  | 100        | 2           |


### 5. `RANK()` і `DENSE_RANK()` — ранжування

```sql
SELECT user_uuid, total_uah,
      RANK() OVER (
          ORDER BY total_uah DESC
      ) AS rank,
      DENSE_RANK() OVER (
          ORDER BY total_uah DESC
      ) AS dense_rank
FROM orders_log;
```

**Різниця:**

* `RANK()` — пропускає ранги (1, 1, 3)
* `DENSE_RANK()` — не пропускає (1, 1, 2)

**Результат:**

| user\_uuid | total\_uah | rank | dense\_rank |
| ---------- | ---------- | ---- | ----------- |
| u2         | 300        | 1    | 1           |
| u1         | 200        | 2    | 2           |
| u1         | 200        | 2    | 2           |
| u1         | 150        | 4    | 3           |
| u2         | 100        | 5    | 4           |
| u1         | 100        | 5    | 4           |


### 6. `FIRST_VALUE()` і `LAST_VALUE()`

```sql
SELECT user_uuid, order_time, total_uah,
      FIRST_VALUE(total_uah) OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      ),
      LAST_VALUE(total_uah) OVER (
         PARTITION BY user_uuid
         ORDER BY order_time
         ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
      )
FROM orders_log;
```

**Що робить:**

* `FIRST_VALUE(...)` — перше значення за заданим `ORDER BY`
* `LAST_VALUE(...)` — останнє значення, потрібно вказати межі вікна

**Результат:**

| user\_uuid | order\_time | total\_uah | first\_value | last\_value |
| ---------- | ----------- | ---------- | ------------ | ----------- |
| u1         | 2024-01-01  | 100        | 100          | 200         |
| u1         | 2024-01-10  | 150        | 100          | 200         |
| u1         | 2024-01-20  | 200        | 100          | 200         |
| u2         | 2024-01-05  | 300        | 300          | 100         |
| u2         | 2024-02-01  | 100        | 300          | 100         |


### 7. `COUNT(*) OVER (PARTITION BY ...)` — кількість замовлень користувача

```sql
SELECT user_id, order_time, total_uah,
       COUNT(*) OVER (
           PARTITION BY user_id
       ) AS orders_cnt
FROM orders_log;
```

**Результат:**

| user_id | order_time | total_uah | orders_cnt |
| ------- | ---------- | --------- | ---------- |
| u1      | 2024-01-01 | 100       | 3          |
| u1      | 2024-01-10 | 150       | 3          |
| u1      | 2024-01-20 | 200       | 3          |
| u2      | 2024-01-05 | 300       | 2          |
| u2      | 2024-02-01 | 100       | 2          |


### 8. Кумулятивна кількість замовлень (`COUNT + ORDER BY`)

```sql
SELECT user_id, order_time, total_uah,
       COUNT(*) OVER (
           PARTITION BY user_id
           ORDER BY order_time
       ) AS cumulative_orders
FROM orders_log;
```

**Результат:**

| user_id | order_time | total_uah | cumulative_orders |
| ------- | ---------- | --------- | ----------------- |
| u1      | 2024-01-01 | 100       | 1                 |
| u1      | 2024-01-10 | 150       | 2                 |
| u1      | 2024-01-20 | 200       | 3                 |
| u2      | 2024-01-05 | 300       | 1                 |
| u2      | 2024-02-01 | 100       | 2                 |


## Задачі

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [2]:
DB_USER = "prog_academy_da_yuq6_user"
DB_PASS = "nKLaf3orGpgat1GEpFbkuNVtxcNAGJp1"
DB_HOST = "dpg-d519e97pm1nc73briqk0-a.oregon-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "prog_academy_da_yuq6"

In [3]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

**Завдання 1. Кумулятивна виручка по кожному користувачу**  
Як накопичувалася сума замовлень у кожного користувача з часом?

**Завдання 2. Інтервал між покупками**  
Скільки днів пройшло з попереднього замовлення?  

*Мета:* Виявити користувачів з довгими перервами — можливо, вони йдуть.

**Завдання 3. Порядковий номер замовлення**  
Яка за рахунком це покупка для користувача?

**Завдання 4. Остання сума покупки (для порівняння)**  
Яка була остання покупка користувача? Чим вона відрізняється від поточної?

**Завдання 5. Найбільше замовлення — на якому місці воно сталося?**  
Найбільше замовлення було першим, другим чи пізніше?  

*Мета:* Зрозуміти, чи є ефект "згасання", або навпаки — зростання довіри.

**Завдання 6. Перше замовлення кожного користувача**  
Яка сума першого замовлення?

# `CASE WHEN`

**`CASE WHEN`** — це умовний оператор у SQL, аналог `if / else`, який дозволяє
повертати різні значення залежно від виконання умов.

Він **не є віконною функцією**, але **дуже часто використовується разом із ними** для:

* створення ознак (features)
* сегментації користувачів
* класифікації подій

---

## Базовий синтаксис

```sql
CASE
    WHEN умова_1 THEN результат_1
    WHEN умова_2 THEN результат_2
    ELSE результат_за_замовчуванням
END
```

📌 Перевірка йде **зверху вниз** — виконується **перша істинна умова**.

---

## Простий приклад

```sql
SELECT total_uah,
       CASE
           WHEN total_uah >= 200 THEN 'high'
           WHEN total_uah >= 100 THEN 'medium'
           ELSE 'low'
       END AS order_size
FROM orders_log;
```

**Логіка:**

* якщо сума ≥ 200 → `high`
* інакше якщо ≥ 100 → `medium`
* інакше → `low`

---

## `CASE WHEN` + віконні функції

Найпоширеніший сценарій — **класифікація на основі результату window-функції**.

### Приклад: перша чи повторна покупка

```sql
SELECT user_uuid, order_time, total_uah,
       CASE
           WHEN ROW_NUMBER() OVER (
                    PARTITION BY user_uuid
                    ORDER BY order_time
                ) > 1 THEN 'R'
           ELSE 'F'
       END AS purchase_type
FROM orders_log;
```

### Як це працює по кроках:

1. `ROW_NUMBER()` нумерує замовлення користувача
2. `CASE WHEN` перевіряє номер:

   * `1` → перша покупка (`F`)
   * `>1` → повторна (`R`)
3. Результат — **новий аналітичний стовпець**

## Обов’язковий `ELSE`

Якщо **не вказати `ELSE`**, SQL поверне `NULL`.

```sql
CASE
    WHEN total_uah > 100 THEN 'big'
END
```

📌 Якщо жодна умова не виконалась → `NULL`

**Рекомендація:** завжди явно вказувати `ELSE`.

---

## Типи даних у `CASE`

Усі `THEN` і `ELSE` мають повертати **сумісні типи**:

❌ Неправильно:

```sql
CASE
    WHEN total_uah > 100 THEN 'big'
    ELSE 0
END
```

✅ Правильно:

```sql
CASE
    WHEN total_uah > 100 THEN 1
    ELSE 0
END
```

або

```sql
CASE
    WHEN total_uah > 100 THEN 'big'
    ELSE 'small'
END
```

---

## Коли використовувати `CASE WHEN`

*  створення сегментів  
*  логічні прапорці (0/1, Y/N, F/R)  
*  категоризація чисел  
*  обробка результатів window-функцій  

**Завдання 7. Ознака повторної покупки**  
Чи зробив користувач щонайменше 2 замовлення?